In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prophet
import datetime
from sklearn.model_selection import GridSearchCV
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.metrics import make_scorer


Df = pd.read_csv('city_combined.csv')
Df.drop('temp_min (⁰C)', axis=1, inplace=True)

Df['windspeed(Kmph)'] = (Df['wind_speed_max (Kmph)'] + Df['wind_speed_min (Kmph)']) / 2
Df['Humidity(%)'] = (Df['Humidity_max (%)'] + Df['humidity_min (%)']) / 2
Df.drop(['wind_speed_min (Kmph)', 'humidity_min (%)', 'Humidity_max (%)', 'wind_speed_max (Kmph)'], axis=1, inplace=True)

Df['windspeed(Kmph)'].fillna(value=Df['windspeed(Kmph)'].mean(), inplace=True)

lockdowns = pd.DataFrame([    {'holiday': 'lockdown_1', 'ds': '2020-03-22', 'lower_window': 0, 'ds_upper': '2020-05-31'},    {'holiday': 'lockdown_2', 'ds': '2021-05-12', 'lower_window': 0, 'ds_upper': '2021-05-22'}])
for t_col in ['ds', 'ds_upper']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days

Df['ds'] = Df['Date']
Df['ds'] = pd.to_datetime(Df['ds'])
Df['y'] = Df['temp_max (⁰C)']
cap = Df['temp_max (⁰C)'].max() + 5
Df['cap'] = cap
Df.drop(["Date", "temp_max (⁰C)"], axis=1, inplace=True)


param_grid = {'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.5], 'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
              'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0]}
prophet_model = prophet.Prophet(growth='logistic', holidays=lockdowns, yearly_seasonality=True)
prophet_model.add_country_holidays(country_name='IN')
for i in ['Rainfall (mm)', 'windspeed(Kmph)', 'Humidity(%)']:
    prophet_model.add_regressor(i)
def custom_scorer(y_true, y_pred):
    df_cv = prophet_model.cross_validation(initial='365 days', period='180 days', horizon='365 days')
    return np.mean(np.square((df_cv.y - df_cv.yhat) / df_cv.y))

prophet_scorer = make_scorer(custom_scorer, greater_is_better=False)

# Run GridSearchCV
grid_search = GridSearchCV(estimator=prophet_model, param_grid=param_grid, scoring=prophet_scorer, cv=5, n_jobs=-1)
grid_search.fit(Df)

best_model = grid_search.best_estimator_

future = best_model.make_future_dataframe(periods=365)
today = [0, 25, 75]
j = 0
for i in ['Rainfall (mm)', 'windspeed(Kmph)', 'Humidity(%)']:
    future[i] = today[j]
    j += 1
future['cap'] = cap
forecast = best_model.predict(future)
forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

cv_results = cross_validation(best_model, initial='730 days', horizon='365 days')
perf_metrics = performance_metrics(cv_results)
print(perf_metrics)


C:\Users\aksha\AppData\Local\Temp\ipykernel_6532\1232341721.py:27: UserWarning: Parsing '13-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_6532\1232341721.py:27: UserWarning: Parsing '14-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_6532\1232341721.py:27: UserWarning: Parsing '15-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_6532\1232341721.py:27: UserWarning: Parsing '16-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_6532

TypeError: Cannot clone object '<prophet.forecaster.Prophet object at 0x0000020AFB31C280>' (type <class 'prophet.forecaster.Prophet'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
def custom_scorer(y_true, y_pred):
    df_cv = m.cross_validation(initial='365 days', period='180 days', horizon='365 days')
    return np.mean(np.square((df_cv.y - df_cv.yhat) / df_cv.y))

prophet_scorer = make_scorer(custom_scorer, greater_is_better=False)

# Run GridSearchCV
grid_search = GridSearchCV(m, param_grid=param_grid, scoring=prophet_scorer, cv=5, n_jobs=-1)